Import packages and setting parameters

In [1]:
# packages
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import os, logging, sys, glob
pd.set_option("display.max_columns", None)

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# --- logging --- #
import logging
logger = logging.getLogger(__name__)
logger.addHandler(logging.StreamHandler())
logger.setLevel(logging.INFO)

# location of the scripts
sys.path.insert(0, '/fefs/aswg/workspace/juan.jimenez/stereo_analysis/scripts')
import auxiliar as aux
import geometry as geom
aux.params()

from astropy.coordinates import AltAz, SkyCoord, EarthLocation, ICRS
from ctapipe.coordinates import CameraFrame
from lstchain.reco.utils import compute_theta2, extract_source_position, clip_alt
from magicctapipe.utils  import calculate_off_coordinates
from astropy import units as u

# --- other parameters --- #
# name of the source we are studying
source_name = 'BLLac'
# number of off regions
n_regions_off  = 5
# if we want to compute the RA and DEC information for the runs that do not have it
# if done, set to False because is along process ~1.5h
compute_ra_dec = True
# ------------------------ #

# --- file paths --- #
root_path = '/fefs/aswg/workspace/juan.jimenez/data'

merged_path  = f'{root_path}/dl2/stereo_merged_{source_name}/*'
mean_path    = f'{root_path}/dl2/stereo_mean/*{source_name}*'
lst_path     = f'{root_path}/dl2/lst/*'
melibea_path = f'{root_path}/dl2/melibea/*'
# ------------------ #

logger.info(f'Study of the source: {source_name}')
logger.info(f'\nAll data taken from the main path {root_path}')
logger.info(f'\n\n--> The selected number of OFF regions is {n_regions_off}')
logger.info(f'\n--> Computing RA and DEC of LST-lstchain files: {compute_ra_dec}')

Study of the source: BLLac

All data taken from the main path /fefs/aswg/workspace/juan.jimenez/data


--> The selected number of OFF regions is 5

--> Computing RA and DEC of LST-lstchain files: True


All the data we have at the directories is:

In [2]:
merged_files = glob.glob(merged_path)
merged_files = [file for file in merged_files if '_to_' not in file]
logger.info(f'\n\nThe merged files are taken from {merged_path}, and {len(merged_files)} are found:')
for f in merged_files:
    logger.info(f'--> {f}')
    
mean_files = glob.glob(mean_path)
logger.info(f'\n\nThe mean files are taken from {mean_path}, and {len(mean_files)} are found:')
for f in mean_files:
    logger.info(f'--> {f}')
    
lst_files = glob.glob(lst_path)
logger.info(f'\n\nThe lst-only files are taken from {lst_path}, and {len(lst_files)} are found:')
for f in lst_files:
    logger.info(f'--> {f}')
    
melibea_files = glob.glob(melibea_path)
logger.info(f'\n\nThe melibea-only files are taken from {melibea_path}, and {len(melibea_files)} are found:')
for f in melibea_files:
    logger.info(f'--> {f}')



The merged files are taken from /fefs/aswg/workspace/juan.jimenez/data/dl2/stereo_merged_BLLac/*, and 7 are found:
--> /fefs/aswg/workspace/juan.jimenez/data/dl2/stereo_merged_BLLac/dl2_merged_BLLac_total.MI_MII.h5
--> /fefs/aswg/workspace/juan.jimenez/data/dl2/stereo_merged_BLLac/dl2_merged_BLLac_MAGIC.3tel.h5
--> /fefs/aswg/workspace/juan.jimenez/data/dl2/stereo_merged_BLLac/dl2_merged_BLLac_total.3tel.h5
--> /fefs/aswg/workspace/juan.jimenez/data/dl2/stereo_merged_BLLac/dl2_merged_BLLac_total.LST1_MI.h5
--> /fefs/aswg/workspace/juan.jimenez/data/dl2/stereo_merged_BLLac/dl2_merged_BLLac_LST.3tel.h5
--> /fefs/aswg/workspace/juan.jimenez/data/dl2/stereo_merged_BLLac/dl2_merged_BLLac_total.all_combo.h5
--> /fefs/aswg/workspace/juan.jimenez/data/dl2/stereo_merged_BLLac/dl2_merged_BLLac_total.LST1_MII.h5


The mean files are taken from /fefs/aswg/workspace/juan.jimenez/data/dl2/stereo_mean/*BLLac*, and 3 are found:
--> /fefs/aswg/workspace/juan.jimenez/data/dl2/stereo_mean/dl2_mean_BLLa

# - Calculus of $\theta^2$ data

First of all for the `lstchain` data we need to compute `ra` and `dec`.

In [ ]:
%%time
if compute_ra_dec == True:

    logger.info('Computing RA and DEC for dl2 LST-lstchain files')
    
    # reading the dataframes
    tmp_mean_dfs    = [pd.read_hdf(file, key='/events/parameters') for file in mean_files]
    tmp_melibea_dfs = [pd.read_hdf(file, key='/events/parameters') for file in melibea_files]
    tmp_lst_dfs     = [pd.read_hdf(file, key='/events/parameters') for file in lst_files]
    
    # iterating over lst ones
    for tmp_df in tmp_lst_dfs:

        # location of LST-1
        location = EarthLocation.from_geodetic(-17.89139 * u.deg, 28.76139 * u.deg, 2184 * u.m) 
        # observation time
        obstime = pd.to_datetime(tmp_df['dragon_time'], unit='s')
        
        # coordinate frame
        horizon_frame = AltAz(location=location, obstime=obstime)    

        # extracting pointing data
        pointing_alt = u.Quantity(tmp_df['alt_tel'], u.rad, copy=False)
        pointing_az  = u.Quantity(tmp_df['az_tel'],  u.rad, copy=False)
        # in alt_az coordinates
        pointing_direction = SkyCoord(alt=clip_alt(pointing_alt), az=pointing_az, frame=horizon_frame)
        # in ra_dec
        pointing_direction_icrs = pointing_direction.transform_to('icrs')
        
        # defining the camera coordinates frame
        camera_frame = CameraFrame(focal_length=28 * u.m,
                                   telescope_pointing=pointing_direction,
                                   obstime=obstime,
                                   location=location)     
        camera_coords = SkyCoord(x=tmp_df['reco_src_x'], y=tmp_df['reco_src_y'], frame=camera_frame, unit=(u.m, u.m))
        radec_coords  = camera_coords.transform_to(frame=ICRS)

        # adding the ra-dec coordinates of pointing and events for lst data
        tmp_df.loc[:,'reco_ra']      = radec_coords.ra.deg
        tmp_df.loc[:,'reco_dec']     = radec_coords.dec.deg 
        tmp_df.loc[:,'pointing_ra']  = pointing_direction_icrs.ra.deg
        tmp_df.loc[:,'pointing_dec'] = pointing_direction_icrs.dec.deg    

        # setting run-event as indexes
        tmp_df.set_index(['obs_id', 'event_id'], inplace=True)
        tmp_df.sort_index(inplace=True)

    # appending all dataframes in a unique array
    all_dfs = [*tmp_mean_dfs, *tmp_melibea_dfs, *tmp_lst_dfs] 

    # deleting temporal dataframes
    del tmp_lst_dfs, tmp_mean_dfs, tmp_melibea_dfs


# only reading the files
else:
    logger.info(f'RA and DEC is already computed so only reading the existing files')
    tmp_mean_dfs = [pd.read_hdf(file,    key='/events/parameters') for file in mean_files]
    tmp_melibea_dfs = [pd.read_hdf(file, key='/events/parameters') for file in melibea_files]
    tmp_lst_dfs = [pd.read_hdf(file,     key='/events/parameters') for file in lst_files]

    # appending all dataframes in a unique array
    all_dfs = [*tmp_mean_dfs, *tmp_melibea_dfs, *tmp_lst_dfs] 
    
    # deleting temporal dataframes
    del tmp_lst_dfs, tmp_mean_dfs, tmp_melibea_dfs

Iterating over all files and computing $\theta^2$ values for `ON` and `OFF` regions. Is defined as, 

$$\theta^2 (\text{ON/OFF})= \text{angular_distance}(\text{event_dir}, \text{ON/OFF_dir})$$

In [ ]:
# empty arrays
all_dfs_with_theta2, all_files_with_theta2 = [], []

# extracting sky coordinates of the source we are studing
on_coord = SkyCoord.from_name(source_name, frame='icrs')
logger.info(f'ON coordinate ({source_name}):\n{on_coord}')
    
# iterate over all dataframes
for file, df in zip(np.concatenate([mean_files, melibea_files, lst_files]), all_dfs):
    logger.info(f'\nFile: {file}')

    # create a figure for each file
    fig, ax = plt.subplots(figsize=(5, 5))
    
    # now extracting off regions
    logger.info(f'\nNumber of OFF regions: {n_regions_off}')
    # loop over every observation ID
    obs_ids = np.unique(df.index.get_level_values('obs_id'))
    for obs_id in obs_ids:

        # filtering dataframe by runs
        df_events = df.query(f'obs_id == {obs_id}')

        # extracting event coordinates
        event_coords = SkyCoord(
            u.Quantity(df_events['reco_ra'], unit='deg'),
            u.Quantity(df_events['reco_dec'], unit='deg'),
            frame='icrs',)
        # extracting mean values
        pnt_ra_mean  = df_events['pointing_ra'].mean()  * u.deg
        pnt_dec_mean = df_events['pointing_dec'].mean() * u.deg

        # plot the pointing mean direction
        ax.scatter(pnt_ra_mean, pnt_dec_mean, marker='x', s=300, color='k', linewidths=1)

        # calculate the angular distances from the ON region
        theta2_on = on_coord.separation(event_coords).to_value('deg') ** 2

        # appending the data to the total dataframe
        df.loc[(obs_id, slice(None)), 'theta2_on'] = theta2_on

        # calculate the OFF coordinates
        off_coords = calculate_off_coordinates(
            pointing_ra=pnt_ra_mean,
            pointing_dec=pnt_dec_mean,
            on_coord_ra=on_coord.ra,
            on_coord_dec=on_coord.dec,
            n_regions=n_regions_off,)

        # iterate over all off regions
        for i_off, off_coord in off_coords.items():

            # calculate the angular distance from the OFF coordinate
            theta2_off = off_coord.separation(event_coords).to_value('deg') ** 2

            # appending the data to the total dataframe
            df.loc[(obs_id, slice(None)), f'theta2_off{i_off}'] = theta2_off

            # plot the OFF coordinate
            ax.scatter(off_coord.ra.to('deg'), off_coord.dec.to('deg'), marker='o', s=500, facecolors='none', edgecolors='grey')

    # legend markers
    ax.scatter([], [], marker='o', s=500, facecolors='none', edgecolors='grey', label='OFF source')   
    ax.scatter([], [], marker='x', s=300, color='k', linewidths=1,              label='Pointing')
    # plot the ON coordinate
    ax.scatter(on_coord.ra.to('deg'), on_coord.dec.to('deg'), label=f'ON, {source_name}', marker='*', s=400, color='deeppink')

    ax.set_xlabel('RA [deg]')
    ax.set_ylabel('Dec [deg]')
    # converting the coordinated to degrees and adding 2 degrees by both sides
    xlim = on_coord.ra.to_value('deg')  + np.array([1, -1])
    ylim = on_coord.dec.to_value('deg') + np.array([-1, 1])
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.grid()
    ax.legend()
    plt.show()
    
    # saving the dataframes and the nam of the file
    all_dfs_with_theta2.append(df.copy())
    all_files_with_theta2.append(file)

Now we save again the dataframes in the respective files, updating it

In [ ]:
for df, file in zip(all_dfs_with_theta2, all_files_with_theta2):
    logger.info(f'Overwritting {file} with theta2 data.\n')
    df.to_hdf(file, key='/events/parameters')

# - Adding more geometrical data and coordinates
- Adding zenith distance, because data is given in `alt` coordinates, we can convert directly using:
$$\text{zd} = 90 - \text{alt}$$

In [ ]:
# puttig all files and data together
all_files = [*merged_files, *mean_files, *melibea_files, *lst_files]
all_dfs   = [pd.read_hdf(file, key='/events/parameters') for file in all_files]

Now we can add the data and change some columns names to maintain consistency of all dataframes.

In [ ]:
for file, df in zip(all_files, all_dfs):
    
    logger.info(f'Writting coordinates for file {file}\n')
    
    # in lst files we rename the variables in order to maintain consistency
    if file in lst_files:
        df.rename({'alt_tel':'pointing_alt'}, axis=1, inplace=True)
        df.rename({'az_tel':'pointing_az'},   axis=1, inplace=True)
        
    # adding pointing direction
    df.loc[:,'pointing_zd'] = df['pointing_alt'] - 90.
    
    # for melibea files we do not have the events information (we have it for both telescopes, 
    # and we do not inport all this information)
    if file not in melibea_files:
        df.loc[:,'reco_zd'] = df['reco_alt'] - 90.
    

Now we save again the dataframes in the respective files, updating it

In [ ]:
for df, file in zip(all_dfs, all_files):
    logger.info(f'Overwritting {file} with zenith distance data.\n')
    df.to_hdf(file, key='/events/parameters')